<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2023/blob/main/Helix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from plotly.subplots import make_subplots
import jax
import jax.numpy as jnp
jax.config.update('jax_enable_x64', True)
from functools import partial

In [2]:
v = np.array([1.,0.]).reshape(2,1)

In [21]:
def rot_mat(theta):
    c=jnp.cos(theta)
    s=jnp.sin(theta)
    return jnp.array([[c, -s],[s, c]])

In [4]:
thetas = np.linspace(0,2*np.pi, 120, endpoint=False)

In [5]:
xys = []
for theta in thetas:
    v_rot = rot_mat(theta) @ v
    xys.append(v_rot)

In [6]:
xys= np.concatenate(xys,axis=1)

In [7]:
fig=make_subplots()
fig.add_scatter(x=xys[0], y=xys[1], mode='markers')
fig.update_layout(width=400, height=400, template='plotly_dark')

In [8]:
np.set_printoptions(precision=4, linewidth=120)

In [9]:
stretchx_matrix=np.array([[2,0],[0,1.]])

In [10]:
stretch_xys = stretchx_matrix @ xys

In [11]:
fig=make_subplots()
fig.add_scatter(x=xys[0], y=xys[1], mode='markers')
fig.add_scatter(x=stretch_xys[0], y=stretch_xys[1], mode='markers')
fig.update_xaxes(range=(-3,3))
fig.update_yaxes(range=(-3,3))
fig.update_layout(width=400, height=400, template='plotly_dark')

In [12]:
rot_stretch_xys= rot_mat(np.pi/6) @ stretch_xys

In [13]:
fig=make_subplots()
fig.add_scatter(x=xys[0], y=xys[1], mode='lines')
fig.add_scatter(x=stretch_xys[0], y=stretch_xys[1], mode='lines')
fig.add_scatter(x=rot_stretch_xys[0], y=rot_stretch_xys[1], mode='lines')
fig.update_xaxes(range=(-3,3))
fig.update_yaxes(range=(-3,3))
fig.update_layout(width=400, height=400, template='plotly_dark')

In [14]:
fig=make_subplots(rows=1,cols=1,specs=[[dict(type='surface')]])
t = np.linspace(0,6*np.pi,1000)
x=np.cos(t)
y=np.sin(t)
z=t/(2*np.pi)
fig.add_scatter3d(x=x,y=y,z=z,mode='lines')
fig.update_layout(width=600,height=600,template='plotly_dark')

In [109]:
R=5.
mz=0.5
a = 1
b= 0.5

def helix(t,R,mz):
    return jnp.array([R*jnp.cos(t), R*jnp.sin(t), mz*t])

In [110]:
def fresnet_serret(f):

    dfdt = jax.jacobian(f)

    def tangent(t):
        dfdt_res = dfdt(t)
        return dfdt_res/jnp.linalg.norm(dfdt_res)

    dtangent_dt = jax.jacobian(tangent)
    def normal(t):
        dtangent_dt_res = dtangent_dt(t)
        return dtangent_dt_res/jnp.linalg.norm(dtangent_dt_res)

    return tangent, normal

In [111]:
tangent, normal=fresnet_serret(lambda t: helix(t, R=R, mz=mz))

In [112]:
def surf(t, theta):
    N = -normal(t)  #N points outwards from helix axis
    T = tangent(t)
    B = jnp.cross(N, T)


    v = rot_mat(theta) @ (jnp.array([1,0]).reshape(2,1))
    return v[0]*a*N + v[1]*b*B + helix(t, R, mz)




In [113]:
surf_vec = jnp.vectorize(surf, signature='(),()->(3)')

In [118]:

nt=200
ntheta=32
ts = jnp.linspace(0,6*jnp.pi, nt)
thetas = jnp.linspace(0,2*jnp.pi, ntheta)

In [119]:
surf_points=surf_vec(ts[:,None], thetas[None,:])

In [120]:
fig=make_subplots(rows=1,cols=1,specs=[[dict(type='surface')]])
t = np.linspace(0,6*np.pi,1000)
x,y,z=helix(t,R,mz)
fig.add_scatter3d(x=x,y=y,z=z,mode='lines')

for t_idx in range(nt):
     x,y,z = surf_points[t_idx,:,:].T
     fig.add_scatter3d(x=x,y=y,z=z,mode='lines')

for theta_idx in range(ntheta):
     x,y,z = surf_points[:,theta_idx,:].T
     fig.add_scatter3d(x=x,y=y,z=z,mode='lines')

fig.update_layout(width=600,height=600,template='plotly_dark')

In [101]:
surf_points.shape

(5, 8, 3)